In [ ]:
# Import necessary libraries
import os
import numpy as np
import matplotlib.pyplot as plt

import torch
import random
from tabulate import tabulate

import dflintdpy.scripts as scripts
from dflintdpy.data.config import HP
from dflintdpy.models.grid import Grid
from dflintdpy.solvers.shortest_path_grb import ShortestPathGrb
from dflintdpy.solvers.symmetric_interdictor import SymmetricInterdictor
from dflintdpy.solvers.asymmetric_interdictor import AsymmetricInterdictor
from dflintdpy.utils.real_world_spni_data_handling import csv_to_graph

from dflintdpy.scripts.compare import (compare_shortest_paths, 
                                       compare_sym_intd,
                                       compare_asym_intd, 
                                       compare_wrong_asym_intd)
from dflintdpy.scripts.setup import (gen_data, 
                                     gen_train_data, 
                                     setup_pfl_predictor, 
                                     setup_dfl_predictor)

## Set Parameters

In [ ]:
seed_number = 0
np.random.seed(seed_number)
seed1, seed2, seed3 = np.random.randint(0, 150, 3).tolist()

In [ ]:
# Initialize the configuration class
cfg = HP()
# Change cfg parameters here
cfg.set("random_seed", seed1)
cfg.set("intd_seed", seed2)
cfg.set("data_loader_seed", seed3)

# cfg.set("num_train_samples", 150)
# cfg.set("num_val_samples", 50)
# cfg.set("num_test_samples", 100)


In [ ]:
# Set the random seed for reproducibility
np.random.seed(cfg.get("random_seed"))
random.seed(cfg.get("random_seed"))
torch.manual_seed(cfg.get("random_seed"))
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(cfg.get("random_seed"))


## Generate Network Data
Includes the network graph, a shortest path opt_model, and training data.

In [ ]:
# Define a graph with appropriate dimensions and an opt_model 
# for solving the shortest path problem on the graph
file_path = os.path.join(
    os.path.dirname(os.getcwd()), 
    'real_world_spni_data/county_level_arcs.csv'
)
# graph = csv_to_graph(file_path)
# cfg.set("grid_size", (graph.num_cost+1, 1))
if True:
    graph = csv_to_graph(file_path)
    cfg.set("grid_size", (graph.num_cost+1, 1))
else:
    m, n = cfg.get("grid_size")
    graph = Grid(m, n)
opt_model = ShortestPathGrb(graph)

# Generate normalized training and testing data
training_data, testing_data, normalization_constant = gen_train_data(cfg, opt_model)

## Predict-then-Optimize

In [ ]:
cfg.set("po_epochs", 150)
cfg.set("spo_epochs", 50)

In [ ]:
po_model = setup_pfl_predictor(
    cfg,
    graph,
    opt_model,
    training_data,
    versatile=True
)

## Smart Predict-then-Optimize with OptNet

In [ ]:
spo_model = setup_dfl_predictor(
    cfg,
    graph,
    opt_model,
    training_data,
    versatile=True
)

## Adverse Predict-then-Optimize with OptNet

In [ ]:
cfg.set("num_scenarios", 1)

In [ ]:
# Generate normalized training and testing data
training_data_non_adverse, testing_data_non_adverse, normalization_constant = gen_train_data(cfg, opt_model)

In [ ]:
spo_model_non_adverse = setup_pfl_predictor(
    cfg,
    graph,
    opt_model,
    training_data_non_adverse,
    versatile=True
)

## Compare PO and SPO
Comparison of performance for solving shortest path problems

In [ ]:
true_objs, po_objs, spo_objs, adv_spo_objs = compare_shortest_paths(
    cfg, opt_model, po_model, spo_model_non_adverse, testing_data, spo_model)

In [ ]:
po_val = [(po - true) / true * 100 for po, true in zip(po_objs, true_objs)]
spo_val = [(spo - true) / true * 100 for spo, true in zip(spo_objs, true_objs)]

fig, ax = plt.subplots(figsize=(6,4))

# Two groups: PO and SPO
_ = ax.boxplot([po_val, spo_val], tick_labels=['PO', 'SPO'],
           showmeans=True, meanline=True)   # optional styling
ax.set_ylabel("Cost increase [%]")
ax.set_ylim(0,250)
ax.grid(True)
# plt.legend(['PO', 'SPO'], location='north')

po_mean  = np.mean(po_val)
spo_mean = np.mean(spo_val)

# std devs
po_std_samp = np.std(po_val, ddof=1)
spo_std_samp = np.std(spo_val, ddof=1)
mean_improvement = (po_mean - spo_mean) * 100 / po_mean
std_improvement = (po_std_samp - spo_std_samp) * 100 / po_std_samp

rows = [
    ["Mean",         f"{po_mean:.2f}%",  f"{spo_mean:.2f}%", f"{mean_improvement:.2f}%"],
    ["Std (sample)", f"{po_std_samp:.2f}%", f"{spo_std_samp:.2f}%", f"{std_improvement:.2f}%"],
]
print(tabulate(rows, headers=["Metric", "PO (%)", "SPO (%)", "SPO improvement (%)"], tablefmt="github"))

## Compare Different Interdictors
We train different prediction models for the interdictors and compare their performance

In [ ]:
interdictions = gen_data(cfg, seed=cfg.get("intd_seed"), normalization_constant=normalization_constant)

In [ ]:
# Comparison of different means of costs to show similar 
print(f"Mean value comparison:")
print(f"\tTest:     {testing_data['costs'].mean():.7f}")
print(f"\tTrain:    {training_data['train_loader'].dataset.costs.mean():.7f}")
print(f"\tIntd:     {interdictions['costs'].mean():.7f}")
print(f"\tPO:       {po_model(torch.tensor(testing_data['feats'], dtype=torch.float32)).mean().item():.7f}")
print(f"\tSPO+:     {spo_model_non_adverse(torch.tensor(testing_data['feats'], dtype=torch.float32)).mean().item():.7f}")
print(f"\tSPO+ adv: {spo_model(torch.tensor(testing_data['feats'], dtype=torch.float32)).std().item():.7f}")

print(f"Std value comparison:")
print(f"\tTest:     {testing_data['costs'].std():.7f}")
print(f"\tTrain:    {training_data['train_loader'].dataset.costs.std():.7f}")
print(f"\tIntd:     {interdictions['costs'].std():.7f}")
print(f"\tPO:       {po_model(torch.tensor(testing_data['feats'], dtype=torch.float32)).std().item():.7f}")
print(f"\tSPO+:     {spo_model_non_adverse(torch.tensor(testing_data['feats'], dtype=torch.float32)).std().item():.7f}")
print(f"\tSPO+ adv: {spo_model(torch.tensor(testing_data['feats'], dtype=torch.float32)).std().item():.7f}")

## Symmetric Interdictions
This interdictor assumes that the evader has full information.

In [ ]:
all_pred_sym_intd = compare_sym_intd(
    cfg, 
    opt_model,
    po_model, 
    spo_model_non_adverse, 
    testing_data, 
    interdictions, 
    normalization_constant, 
    adfl_predictor=spo_model
)

## Asymmetric interdictions with correct evader model

In [ ]:
no_pred_asym_intd = compare_asym_intd(
    cfg, 
    opt_model,
    testing_data, 
    interdictions, 
    normalization_constant,
)

In [ ]:
po_pred_asym_intd_I = compare_asym_intd(
    cfg, 
    opt_model,
    testing_data, 
    interdictions, 
    normalization_constant, 
    po_model
)

In [ ]:
spo_pred_asym_intd_I = compare_asym_intd(
    cfg, 
    opt_model,
    testing_data, 
    interdictions, 
    normalization_constant, 
    spo_model_non_adverse
)

In [ ]:
adv_spo_pred_asym_intd_I = compare_asym_intd(
    cfg, 
    opt_model,
    testing_data, 
    interdictions, 
    normalization_constant, 
    spo_model
)

## Asymmetric interdictions with wrong evader models

In [ ]:
true_nonadv_false_po_asym_intd = compare_wrong_asym_intd(
    cfg, 
    opt_model,
    testing_data, 
    interdictions, 
    normalization_constant, 
    true_model=spo_model_non_adverse, 
    false_model=po_model
)

In [ ]:
true_po_false_nonadv_asym_intd = compare_wrong_asym_intd(
    cfg, 
    opt_model,
    testing_data, 
    interdictions, 
    normalization_constant, 
    true_model=po_model, 
    false_model=spo_model_non_adverse
)

In [ ]:
true_spo_false_po_asym_intd = compare_wrong_asym_intd(
    cfg, 
    opt_model,
    testing_data, 
    interdictions, 
    normalization_constant, 
    true_model=spo_model, 
    false_model=po_model
)

In [ ]:
true_po_false_spo_asym_intd = compare_wrong_asym_intd(
    cfg, 
    opt_model,
    testing_data, 
    interdictions, 
    normalization_constant, 
    true_model=po_model, 
    false_model=spo_model
)

In [ ]:
true_adv_false_nonadv_asym_intd = compare_wrong_asym_intd(
    cfg, 
    opt_model,
    testing_data, 
    interdictions, 
    normalization_constant, 
    true_model=spo_model, 
    false_model=spo_model_non_adverse
)

In [ ]:
true_nonadv_false_adv_asym_intd = compare_wrong_asym_intd(
    cfg, 
    opt_model,
    testing_data, 
    interdictions, 
    normalization_constant, 
    true_model=spo_model_non_adverse, 
    false_model=spo_model
)

## Sanity checks and results

In [ ]:
print(f"Evader Adverse, Intd Nonadverse: {true_adv_false_nonadv_asym_intd.mean()} +/- {true_adv_false_nonadv_asym_intd.std()}")
print(f"Evader Nonadverse, Intd Adverse: {true_nonadv_false_adv_asym_intd.mean()} +/- {true_nonadv_false_adv_asym_intd.std()}")

In [ ]:
eps = 1e-5  # Define a small epsilon for numerical stability
print(f"Instances where Sym is better than Asym: {sum([1 if sym - asym > eps else 0 for sym, asym in zip(all_pred_sym_intd['spo_objective'], spo_pred_asym_intd_I)]) / len(po_objs)}")
# print(f"Instances where SPO is better than PO: {sum([1 if po - spo > eps else 0 for po, spo in zip(all_pred_sym_intd['spo_objective'], spo_pred_asym_intd)]) / len(po_objs)}")
# print(f"Sanity check (equal performance):      {sum([1 if abs(po - spo) < eps else 0 for po, spo in zip(all_pred_sym_intd['spo_objective'], spo_pred_asym_intd)]) / len(po_objs)}")
# Interesting result: The percentage of PO better than SPO is almost equal to SPO better than PO.

In [ ]:
# [(asym - sym).item() for sym, asym in zip(all_pred_sym_intd['spo_objective'], spo_pred_asym_intd)]
# print(all_pred_sym_intd['true_objective'][0])
# print(no_pred_asym_intd[0])
# testing_data['costs'].std()
# sym is better than asym in 28% of instances without calibration

# Prepare no-interdiction results for printing
true_mean = np.array(true_objs).mean() * normalization_constant
po_mean = np.array(po_objs).mean() * normalization_constant
spo_mean = np.array(spo_objs).mean() * normalization_constant
adv_spo_mean = np.array(adv_spo_objs).mean() * normalization_constant

In [ ]:
print(f"DFL no intd. improvement = {po_mean - spo_mean:.2f}")
print(f"Adv. DFL no intd. improvement = {po_mean - adv_spo_mean:.2f}")
print(f"Adv. DFL sym. improvement = {all_pred_sym_intd['po_objective'].mean() - all_pred_sym_intd['adv_spo_objective'].mean():.2f}")
print(f"Adv. DFL asym. improvement = {po_pred_asym_intd_I.mean() - adv_spo_pred_asym_intd_I.mean():.2f}")
print(f"PO Asym. + Adv. Evader > Sym Asym. = {true_po_false_spo_asym_intd.mean() - all_pred_sym_intd['adv_spo_objective'].mean():.2f}")

In [ ]:
# Precentage improvements, read as: 
# <prediction type>_<original interdiction>_<changed interdiction> 
# -> percentage is improvement from the original interdiction to the changed interdiction
# po_no_sym = (po_pred_sym_intd-po_pred_no_intd)/po_pred_no_intd
# po_no_asym = (po_pred_asym_intd-po_pred_no_intd)/po_pred_no_intd
# spo_no_sym = (spo_pred_sym_intd-spo_pred_no_intd)/spo_pred_no_intd
# spo_no_asym = (spo_pred_asym_intd-spo_pred_no_intd)/spo_pred_no_intd

# Percentage improvements, read as:
# <original prediction type>_<changed prediction type>_<interdiction type>
# -> percentage is improvement from the original prediction type to the changed prediction type
# po_spo_no = (spo_pred_no_intd-po_pred_no_intd)/po_pred_no_intd
# po_spo_sym = (spo_pred_sym_intd-po_pred_sym_intd)/po_pred_sym_intd
# po_spo_asym = (spo_pred_asym_intd-po_pred_asym_intd)/po_pred_asym_intd

# Prepare no-interdiction results for printing
true_mean = np.array(true_objs).mean() * normalization_constant
po_mean = np.array(po_objs).mean() * normalization_constant
spo_mean = np.array(spo_objs).mean() * normalization_constant
adv_spo_mean = np.array(adv_spo_objs).mean() * normalization_constant

# Print the results in a table format
table_headers = ["Predictor", "No Interdictor", "Sym. Interdictor", "Asym. Interdictor", "Asym. Intd. Assumes PO", "Asym. Intd. Assumes SPO", "Asym. Intd Assumes Adv. SPO"]

rows = [
    [
        "Oracle", 
        f"{true_mean:.4f}", 
        f"{all_pred_sym_intd['true_objective'].mean():.4f} +/- {all_pred_sym_intd['true_objective'].std():.4f}", 
        f"{no_pred_asym_intd.mean():.4f} +/- {no_pred_asym_intd.std():.4f}", 
        # "N/A", 
        # "N/A",
        # "N/A"
    ], [
        "PO", 
        f"{po_mean:.4f} ",  
        f"{all_pred_sym_intd['po_objective'].mean():.4f} +/- {all_pred_sym_intd['po_objective'].std():.4f}", 
        f"{po_pred_asym_intd_I.mean():.4f} +/- {po_pred_asym_intd_I.std():.4f}", 
        # "", 
        # f"{true_po_false_nonadv_asym_intd.mean():.4f} +/- {true_po_false_nonadv_asym_intd.std():.4f}",
        # f"{true_po_false_spo_asym_intd.mean():.4f} +/- {true_po_false_spo_asym_intd.std():.4f}"
    ], [
        "SPO", 
        f"{spo_mean:.4f} ", 
        f"{all_pred_sym_intd['spo_objective'].mean():.4f} +/- {all_pred_sym_intd['spo_objective'].std():.4f}", 
        f"{spo_pred_asym_intd_I.mean():.4f} +/- {spo_pred_asym_intd_I.std():.4f}", 
        # f"{true_nonadv_false_po_asym_intd.mean():.4f} +/- {true_nonadv_false_po_asym_intd.std():.4f}",
        # "",
        # f"{true_nonadv_false_adv_asym_intd.mean():.4f} +/- {true_nonadv_false_adv_asym_intd.std():.4f}", 
    ], [
        "SPO adv", 
        f"{adv_spo_mean:.4f} ", 
        f"{all_pred_sym_intd['adv_spo_objective'].mean():.4f} +/- {all_pred_sym_intd['adv_spo_objective'].std():.4f}", 
        f"{adv_spo_pred_asym_intd_I.mean():.4f} +/- {adv_spo_pred_asym_intd_I.std():.4f}", 
        # f"{true_spo_false_po_asym_intd.mean():.4f} +/- {true_spo_false_po_asym_intd.std():.4f}", 
        # f"{true_adv_false_nonadv_asym_intd.mean():.4f} +/- {true_adv_false_nonadv_asym_intd.std():.4f}",
        # ""
    ]
]
print(tabulate(rows, headers=table_headers, tablefmt="github"))

print("\n")

table_headers = ["Predictor", "Asym. Intd. Assumes PO", "Asym. Intd. Assumes SPO", "Asym. Intd Assumes Adv. SPO"]

rows = [
    [
        "Oracle", 
        # f"{true_mean:.4f}", 
        # f"{all_pred_sym_intd['true_objective'].mean():.4f} +/- {all_pred_sym_intd['true_objective'].std():.4f}", 
        # f"{no_pred_asym_intd.mean():.4f} +/- {no_pred_asym_intd.std():.4f}", 
        "N/A", 
        "N/A",
        "N/A"
    ], [
        "PO", 
        # f"{po_mean:.4f} ",  
        # f"{all_pred_sym_intd['po_objective'].mean():.4f} +/- {all_pred_sym_intd['po_objective'].std():.4f}", 
        # f"{po_pred_asym_intd_I.mean():.4f} +/- {po_pred_asym_intd_I.std():.4f}", 
        "", 
        f"{true_po_false_nonadv_asym_intd.mean():.4f} +/- {true_po_false_nonadv_asym_intd.std():.4f}",
        f"{true_po_false_spo_asym_intd.mean():.4f} +/- {true_po_false_spo_asym_intd.std():.4f}"
    ], [
        "SPO", 
        # f"{spo_mean:.4f} ", 
        # f"{all_pred_sym_intd['spo_objective'].mean():.4f} +/- {all_pred_sym_intd['spo_objective'].std():.4f}", 
        # f"{spo_pred_asym_intd_I.mean():.4f} +/- {spo_pred_asym_intd_I.std():.4f}", 
        f"{true_nonadv_false_po_asym_intd.mean():.4f} +/- {true_nonadv_false_po_asym_intd.std():.4f}",
        "",
        f"{true_nonadv_false_adv_asym_intd.mean():.4f} +/- {true_nonadv_false_adv_asym_intd.std():.4f}", 
    ], [
        "SPO adv", 
        # f"{adv_spo_mean:.4f} ", 
        # f"{all_pred_sym_intd['adv_spo_objective'].mean():.4f} +/- {all_pred_sym_intd['adv_spo_objective'].std():.4f}", 
        # f"{adv_spo_pred_asym_intd_I.mean():.4f} +/- {adv_spo_pred_asym_intd_I.std():.4f}", 
        f"{true_spo_false_po_asym_intd.mean():.4f} +/- {true_spo_false_po_asym_intd.std():.4f}", 
        f"{true_adv_false_nonadv_asym_intd.mean():.4f} +/- {true_adv_false_nonadv_asym_intd.std():.4f}",
        ""
    ]
]
print(tabulate(rows, headers=table_headers, tablefmt="github"))

# Boxplot of the true objectives for both interdiction methods
fig, ax = plt.subplots(figsize=(6,4))
_ = ax.boxplot([all_pred_sym_intd["po_objective"], all_pred_sym_intd["spo_objective"], po_pred_asym_intd_I, spo_pred_asym_intd_I],
           tick_labels=['Symmetric PO', 'Symmetric SPO', 'Asymmetric PO', 'Asymmetric SPO'],
           showmeans=True, meanline=True)
ax.set_ylabel("Interdicted Shortest Path Length")
ax.grid(True)